In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [71]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import KFold

In [33]:
titanic_df = pd.read_csv('train.csv')

In [34]:
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
titanic_df['Age'].sum()

21205.17

In [27]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,373450,8.0500,S


In [35]:
titanic_df = titanic_df.drop('Cabin' , axis=1) #删除掉Cabin column

In [36]:
titanic_df = titanic_df.drop('Parch' , axis = 1) #删除掉 Parch column

In [37]:
titanic_df['Age'] = titanic_df.Age.fillna(titanic_df['Age'].mean() ) #使用平均值 对年龄进行填充

In [38]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [42]:
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')

In [44]:
titanic_df.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [45]:
sex_map = {'male':1 , 'female':2}
embarked_map = {'S':1 , 'C':2 , 'Q':3}

titanic_df['Sex'] = titanic_df.Sex.map(sex_map)
titanic_df['Embarked'] = titanic_df.Embarked.map(embarked_map)

In [85]:
titanic_df['Name'] = titanic_df.Name.str.extract(' ([A-Za-z]+)\.') #将名字中的 称呼 字段提取出来

temp = titanic_df.Name.value_counts()
name_map = {}
i=0

for key in temp.index:
    name_map[key] = i
    i=i+1

titanic_df['Name'] = titanic_df['Name'].map(name_map)

In [46]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Embarked       891 non-null int64
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB


In [93]:
kf = KFold(n_splits=10 , shuffle=True)

In [94]:
model = DecisionTreeClassifier()

In [98]:
scores = 0.0
for train , test in kf.split(titanic_df):
    train_data = titanic_df.loc[train , ['Name' , 'Pclass','Sex' , 'Age' , 'Fare' , 'Embarked']]
    test_data = titanic_df.loc[test , ['Name','Pclass','Sex' , 'Age' , 'Fare' , 'Embarked']]
    
    model.fit(train_data , titanic_df.loc[train , 'Survived'])
    score = model.score(test_data , titanic_df.loc[test , 'Survived'])
    scores += score
    print(score)

print('average:', scores/10)

0.744444444444
0.752808988764
0.775280898876
0.752808988764
0.797752808989
0.85393258427
0.719101123596
0.797752808989
0.808988764045
0.76404494382
average: 0.776691635456
